In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
data_path = "\NMT\\spa-eng\\spa.txt"

In [3]:
#Read the data
lines= pd.read_table(data_path,names=['source', 'target', 'comments'])
lines.sample(5)

,source,target,comments
31839,Do you feel better now?,¿Te sentís mejor ahora?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
39182,Tom clicked his fingers.,Tomás se chupó los dedos.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
46821,It's just how things work.,Es solo como funcionan las cosas.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
50220,I gave my old clothes away.,Regalé mi ropa vieja.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
113299,I am thirsty. I would like to have a cup of co...,Tengo sed. Quisiera una taza de café.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [4]:
#Clean the data
# Converts source and traget to lower case
lines.source= lines.source.apply(lambda x :x.lower())
lines.target= lines.target.apply(lambda x :x.lower())

In [5]:
# remove special charcaters
special_characters= set(string.punctuation)
print(special_characters)
lines.source = lines.source.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
lines.target = lines.target.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
lines.sample(2)

{'<', '{', '=', '!', '&', '@', '.', '[', '$', '~', '_', '%', '"', ')', ']', '#', '\\', '`', '}', '>', '+', '|', '*', '?', '/', '-', '(', ',', ';', '^', "'", ':'}


,source,target,comments
40856,have i made myself clear,¿me he explicado con claridad,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
122785,tom and mary dont have much time to talk toget...,tom y mary no tienen mucho tiempo para hablar ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [6]:
#remove quotes
lines.source= lines.source.apply(lambda x : re.sub("'", '', x))
lines.target = lines.target.apply(lambda x :re.sub("'", '', x))
lines.sample(3)

,source,target,comments
18670,tom died in prison,tom murió en prisión,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
113659,the city hall is located at the center of the ...,el ayuntamiento se encuentra en el centro de l...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
8815,it didnt break,no se rompió,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [7]:
#Remove digits
num_digits= str.maketrans('','', digits)
print(num_digits)
lines.source=lines.source.apply(lambda x: x.translate(num_digits))
lines.target= lines.target.apply(lambda x: x.translate(num_digits))
lines.sample(3)

{48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None}


,source,target,comments
88169,i argued with him about the new plan,discutí con él acerca del nuevo plan,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
51102,life is like a soap bubble,la vida es como una burbuja de jabón,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
19258,what happened here,¿qué pasó aquí,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [8]:
#Remove spaces
lines.source = lines.source.apply(lambda x:x.strip())
lines.target= lines.target.apply(lambda x: x.strip())
lines.sample(10)


,source,target,comments
44003,tom was marys stepchild,tom era el hijastro de mary,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
95727,he gazed at the ceiling for a long time,estuvo mirando el cielo raso durante un largo ...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
67751,im too poor to buy a new suit,soy demasiado pobre para comprar un traje nuevo,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
80602,tom explained the project to mary,tom le explicó el proyecto a mary,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
43623,tom didnt get it either,tom no lo consiguió tampoco,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
114169,how did you find out that tom hadnt paid his b...,¿cómo averiguaste que tom no había pagado sus ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
120183,now that you are grownup it is up to you to de...,ahora que eres mayor decidir qué hacer depende...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
8443,i need a weapon,necesito un arma,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
37815,ill expect you at,te esperaré a las dos y media,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
28139,he left the door open,él dejó la puerta abierta,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [9]:
#remove "+"
lines.source= lines.source.apply(lambda x :re.sub(" +" ," ", x))
lines.target = lines.target.apply(lambda x :re.sub(" +", " ", x))
lines.sample(3)

,source,target,comments
50912,is all of this money yours,¿es tuyo todo este dinero,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
39228,tom felt like a new man,tom se sintió como un hombre nuevo,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
55887,tell me where you were born,dime dónde naciste,CC-BY 2.0 (France) Attribution: tatoeba.org #3...


In [10]:
#Add <Start and <END> tags to the target
lines.target= lines.target.apply(lambda x: 'START_ ' + x + ' _END')
lines.sample(20)

,source,target,comments
104701,the bus was late because of the traffic jam,START_ el autobús llegó tarde por el atasco _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
50177,i dont see any difference,START_ no veo ninguna diferencia _END,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
15383,tom is my student,START_ tom es mi alumno _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
38539,she hates green peppers,START_ ella odia los pimientos verdes _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
35721,who discovered america,START_ ¿quién descubrió américa _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
92468,tom has many friends living in boston,START_ tom tiene muchos amigos viviendo en bos...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
4919,its my fault,START_ la culpa es mía _END,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
39362,tom is saying something,START_ tom está diciendo algo _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3999,burn the body,START_ quema el cuerpo _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
52621,we apologize for the delay,START_ nos disculpamos por la demora _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [11]:
# Find all the source and target words and sort them 
all_source_words= set()
for line in lines.source:
    for word in line.split():
        if word not in all_source_words:
            all_source_words.add(word)
source_words= sorted(list(all_source_words))
print(source_words[:10])
all_target_words=set()
for line in lines.target:
    for word in line.split():
        if word not in all_target_words:
            all_target_words.add(word)
target_words=sorted(list(all_target_words))
print(target_words[:10])                   

['a', 'aardvark', 'aardvarks', 'aaron', 'aback', 'abandon', 'abandoned', 'abandoning', 'abate', 'abated']
['START_', '_END', 'a', 'aabe', 'aah', 'aaron', 'abajo', 'abandona', 'abandonada', 'abandonadas']


In [12]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.source:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(max_source_length)

target_length_list=[]
for l in lines.target:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(max_target_length)

47
55


In [13]:
# create a word2idxfor source and target words
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])
                       
                                                        
#print(source_word2idx)
#print(target_word2idx)

In [14]:
# create index to word for source and traget words
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])
#print(source_idx2word)
#print(target_idx2word)

In [15]:
# Input tokens for encoder
num_encoder_tokens=len(source_words)
# Input tokens for encoder
num_decoder_tokens=len(target_words) +1 #zero paddind
print(num_encoder_tokens)
print(num_decoder_tokens)

13467
27200


In [16]:
#print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_source_length)
print('Max sequence length for outputs:', max_target_length)

Number of unique input tokens: 13467
Number of unique output tokens: 27200
Max sequence length for inputs: 47
Max sequence length for outputs: 55


In [17]:
#Shuffling data serves the purpose of reducing variance and making sure that models remain general and overfit less.
lines = shuffle(lines)
lines.head(10)

,source,target,comments
60387,the bowl was perfectly round,START_ el cuenco era perfectamente redondo _END,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
89444,the storm showed no signs of abating,START_ la tormenta no mostró señales de amaina...,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
102866,she hasnt been to the dentists for years,START_ ella no ha ido al dentista por años _END,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
27347,you can put it there,START_ lo puede dejar ahí _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
66180,do you guys have any more milk,START_ ¿os queda leche _END,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
76912,tom blames mary for johns death,START_ tom culpa a maría por la muerte de juan...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
20723,i enjoy a challenge,START_ yo aprecio un desafío _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
121813,i think its highly unlikely that well be able ...,START_ creo que es muy improbable que podamos ...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
85555,i need to know what happened to tom,START_ necesito saber qué le sucedió a tom _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
41820,i often get things wrong,START_ a menudo no hago las cosas bien _END,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [18]:
# Train - Test Split
X, y = lines.source, lines.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((110642,), (12294,))

In [19]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = source_word2idx[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [20]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 25
latent_dim=50

In [21]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


Instructions for updating:
Colocations handled automatically by placer.


In [22]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     673350      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     1360000     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [24]:
model.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(13467, 50) dtype=float32_ref>,
 <tf.Variable 'embedding_2/embeddings:0' shape=(27200, 50) dtype=float32_ref>,
 <tf.Variable 'lstm_1/kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_1/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'lstm_2/kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_2/recurrent_kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_2/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(50, 27200) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(27200,) dtype=float32_ref>]

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

In [ ]:
model.save_weights('nmt_weights_pad100_nmt_tech2.h5')

In [43]:
model.load_weights('nmt_weights_nopad100.h5')

In [44]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]


# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)

In [45]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [46]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [47]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: dont say that
Actual Target Translation:  no digas eso 
Predicted Target Translation:  no lo que te pasó 


In [48]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [50]:
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])

Input Source sentence: i have never felt better
Actual Target Translation:  nunca me sentí mejor 
Predicted Target Translation:  nunca me sentí mejor 
